In [1]:
%tensorflow_version 1.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')
else:
  print("GPU found")

import pandas as pd
import numpy as np
import os 

After that, `%tensorflow_version 1.x` will throw an error.

Your notebook should be updated to use Tensorflow 2.
See the guide at https://www.tensorflow.org/guide/migrate#migrate-from-tensorflow-1x-to-tensorflow-2.

TensorFlow 1.x selected.
GPU found


In [2]:
def extract_users_items_predictions(data_pd):
    users, movies = \
        [np.squeeze(arr) for arr in np.split(data_pd.Id.str.extract('r(\d+)_c(\d+)').values.astype(int) - 1, 2, axis=-1)]
    predictions = data_pd.Prediction.values
    return users, movies, predictions

In [3]:
x_train_pd = pd.read_csv('drive/MyDrive/CIL Project/data_train.csv')

In [4]:
users,movies,predictions = extract_users_items_predictions(x_train_pd)
ratings_dict = {'userID': users,'movieID': movies,'rating': predictions}
df_train = pd.DataFrame(ratings_dict) 

In [5]:
DATA_DIR = '/content/drive/MyDrive/CIL Project/kernelNet'

In [6]:
np.savetxt(os.path.join(DATA_DIR, "ratings.dat"), df_train.values, delimiter='::', fmt='%s',encoding='utf-8')

In [ ]:
!python '/content/drive/MyDrive/CIL Project/kernelNet/kernelNet_ml1m.py' 30 0.05

reading data...
data read in 5.838801145553589 seconds
loaded dense data matrix



The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



2022-07-14 18:03:30.468660: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-07-14 18:03:30.474462: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-14 18:03:30.475038: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Found devi

In [8]:
#load entries to predict 
to_predict_pd = pd.read_csv('drive/MyDrive/CIL Project/sampleSubmission.csv')
pred_users,pred_movies,pred_predictions = extract_users_items_predictions(to_predict_pd)
to_predict_dict = {'userID': pred_users,'movieID': pred_movies,'rating': pred_predictions}
df_predict = pd.DataFrame(to_predict_dict) 
np.savetxt(os.path.join(DATA_DIR, "to_predict.dat"), df_predict.values, delimiter='::', fmt='%s',encoding='utf-8')

In [9]:
# i can throw  0'th column of raw predictions away 
raw_predictions = pd.read_csv("/content/drive/MyDrive/CIL Project/kernelNet/raw_predictions.csv")

In [10]:
output = to_predict_pd.to_numpy()
final_predictions = raw_predictions.to_numpy()
for id,user in enumerate(pred_users):
  prediction = final_predictions[pred_movies[id]][user+1]
  output[id][1] = prediction 
print(output)

[['r37_c1' 3.3437703]
 ['r73_c1' 3.1270108]
 ['r156_c1' 3.8019218]
 ...
 ['r9978_c1000' 2.9503036]
 ['r9982_c1000' 3.3969204]
 ['r9996_c1000' 3.7436376]]


In [11]:
submission_df = pd.DataFrame(output, columns = ['Id', 'Prediction'])
print(submission_df)
submission_df.to_csv('kernelNet.csv',index = False)

                  Id Prediction
0             r37_c1    3.34377
1             r73_c1   3.127011
2            r156_c1   3.801922
3            r160_c1   3.356608
4            r248_c1   3.560508
...              ...        ...
1176947  r9974_c1000   3.652831
1176948  r9977_c1000    3.22678
1176949  r9978_c1000   2.950304
1176950  r9982_c1000    3.39692
1176951  r9996_c1000   3.743638

[1176952 rows x 2 columns]


In [12]:
print(np.max(pred_users))
print(np.max(pred_movies))

9999
999


In [13]:
print(raw_predictions.shape)

(1000, 10001)


In [14]:
print(raw_predictions.iloc[:,0])

0        0
1        1
2        2
3        3
4        4
      ... 
995    995
996    996
997    997
998    998
999    999
Name: Unnamed: 0, Length: 1000, dtype: int64


In [15]:
print(raw_predictions.head())

   Unnamed: 0         0         1         2         3         4         5  \
0           0  3.298078  3.364606  3.107945  3.372899  3.346610  3.968790   
1           1  3.573898  3.582423  3.667264  3.493652  3.604691  4.173111   
2           2  3.449914  2.476642  3.032557  3.087402  3.049208  3.593947   
3           3  3.858769  3.391462  3.749388  3.383215  4.166865  4.398826   
4           4  3.435986  3.057750  3.195803  3.351379  3.436681  3.973326   

          6         7         8  ...      9990      9991      9992      9993  \
0  3.393194  3.231439  3.511101  ...  3.310962  3.399398  2.470997  3.308968   
1  3.919772  3.626485  3.548094  ...  3.685584  3.937322  2.845666  3.760041   
2  3.168086  3.455462  3.507564  ...  2.856803  3.546296  2.472562  2.968100   
3  4.577912  4.565358  4.640500  ...  3.559577  3.942229  3.332286  4.267271   
4  3.964025  3.777218  4.248209  ...  3.325190  3.376357  2.613126  3.728202   

       9994      9995      9996      9997      9998     